In [0]:
import pandas as pd
path = "/Workspace/Users/johnkim9597@gmail.com/data/telco.csv"
pdf = pd.read_csv(path)
df = spark.createDataFrame(pdf)
display(df)


In [0]:
# 1.Create a dataframe containing 'customerID' and 'MonthlyCharges'
df_selected = df.select("customerID", "MonthlyCharges")
display(df_selected)

In [0]:
# 2.Show on a pie chart the proportions of possible types of 'Contract'
# Count by the Contract
contract_counts = df.groupBy("Contract").count()

display(contract_counts)

Databricks visualization. Run in Databricks to view.

In [0]:
# 3.Get the number of customers that have Multiple Lines. Show the result as value of a single variable
# 'MultipleLines'가 'Yes'인 고객 필터링 후 개수 세기
from pyspark.sql.functions import col
num_multiple_lines = df.filter(col("MultipleLines") == "Yes").count()

print(f"Number of customers with Multiple Lines: {num_multiple_lines}")

In [0]:
# 4.What are the minimum and maximum values of tenure? Show these as single variables. 
from pyspark.sql.functions import min, max

# min, max 계산
tenure_stats = df.select(min("tenure"), max("tenure")).collect()

min_tenure = tenure_stats[0][0]
max_tenure = tenure_stats[0][1]

print(f"Min tenure: {min_tenure}, Max tenure: {max_tenure}")

In [0]:
# 5.What are the monthly charges depending on the gender of the customer?
from pyspark.sql.functions import avg

gender_charges = df.groupBy("gender").agg(avg("MonthlyCharges").alias("AvgMonthlyCharges"))
display(gender_charges)

In [0]:
# 6. Do customers who have a yearly contract pay on average more than the others? Return the result as a single bool variable. Please note that others in this context means that we treat together both the Two year and month-to-month customers.

# 1. One year Contract avg
avg_yearly = df.filter(col("Contract") == "One year").select(avg("MonthlyCharges")).collect()[0][0]

# 2. Others avg
avg_others = df.filter(col("Contract") != "One year").select(avg("MonthlyCharges")).collect()[0][0]

# 3. bool 
is_yearly_higher = avg_yearly > avg_others

print(f"One year average: {avg_yearly}")
print(f"Others average: {avg_others}")
print(f"Is yearly contract average higher? {is_yearly_higher}")

In [0]:
#7. Create a column that will show the average charges computed as a division of Total Charges by tenure
from pyspark.sql.functions import col, when, trim
# TotalCharges -> double (trim)
df_cleaned = df.withColumn("TotalCharges", 
    when(trim(col("TotalCharges")) == "", None)
    .otherwise(trim(col("TotalCharges")))
    .cast("double")
)

df_final = df_cleaned.withColumn("Avg_Charges_By_Tenure", col("TotalCharges") / col("tenure"))

display(df_final.select("customerID", "TotalCharges", "tenure", "Avg_Charges_By_Tenure"))